<a href="https://colab.research.google.com/github/kasier48/DeepLearning/blob/main/Pratice_Week4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [4주차] 심화과제: 수능 국어 문제 GPT-4로 풀어보기

이번 과제에서는 2023년도 수능 국어 문제를 GPT-4로 풀어볼 것입니다. 아래 요구사항들을 지켜주시면 됩니다.

- [ ]  수능 국어 문제를 준비합니다. 다음 github의 `data > 2023_11_KICE.json` data를 colab으로 불러오시면 됩니다:
    
    [GitHub - NomaDamas/KICE_slayer_AI_Korean: 수능 국어 1등급에 도전하는 AI](https://github.com/NomaDamas/KICE_slayer_AI_Korean)
    
- [ ]  하나의 문제에 대해서 GPT-4의 예측 결과를 내놓는 함수를 `def prediction(problem)`이라는 signature로 만드셔야 합니다. `problem`은 json 형태의 문제입니다. 내부는 logit 계산을 통해 구현하거나 순수하게 text 생성으로 해결하셔도 좋습니다. ***단, 2023년도 수능 국어의 정답을 활용하시면 안됩니다.***
- [ ]  `def prediction` 함수를 모든 수능 국어 문제들에 대해서 돌린 후, 실제 정답과 비교하여 GPT-4의 점수를 계산하는 코드를 구현하시면 됩니다. ***단, 점수 계산은 모두 코드를 통해서만 진행되어야 합니다.*** 사람이 직접 GPT-4의 출력 결과를 보고 대조하는 형식으로 되면 안됩니다.
- [ ]  채점 결과 80점을 넘기면 통과입니다.

GPT API 사용 및 GPT-4o를 사용해야 한다는 것 이외에는 제약사항이 없습니다.

In [ ]:
!pip install openai datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from openai import OpenAI
from google.colab import userdata
import json

# [MYCODE] openai_key 설정
openai_key = userdata.get('')
def prediction(user_prompt):
  client = OpenAI(api_key=openai_key)

  temperature = 0.5
  max_tokens = 4096
  frequency_penalty = 0.0

  # [MYCODE] asistant_prompt로 힌트 제공.
  assistant_prompt = "당신은 한국어에 능통한 사람 입니다. 가장 정답이 높은 번호를 선택해주세요. 답변은 오직 정답인 숫자만 해주세요. 정답은 오직 하나 입니다."

  message=[{"role": "assistant", "content": assistant_prompt}, {"role": "user", "content": user_prompt}]
  response = client.chat.completions.create(
      model="gpt-4o",
      messages=message,
      n=1,
      max_tokens=max_tokens,
      temperature=temperature,
      frequency_penalty=frequency_penalty
  )
  answer = response.choices[0].message.content
  print(f'gtp가 추론한 정답: {answer}')

  return answer

# [MYCODE] 정답이 틀릴 경우 틀린 이유를 분석해서 다시 선택하도록 유도.
def predict_again(question, uncorrect_answer):
  client = OpenAI(api_key=openai_key)

  temperature = 0.5
  max_tokens = 4096
  frequency_penalty = 0.0

  # [MYCODE] assistent로 어떤 문제인지 알려줌.
  assistant_prompt = f"문제는 다음과 같습니다. {question}"

  # [MYCODE] user로 오답이 무엇인지 알려주고 틀린 이유를 분석 하도록 한 이후에 다시 정답을 선택하도록 함.
  user_prompt = f"당신은 오답 {uncorrect_answer}번을 선택하였습니다. 틀린 이유를 분석하고 정답을 다시 선택해주세요. 대답은 숫자로만 해주세요."
  message=[{"role": "assistant", "content": assistant_prompt},{"role": "user", "content": user_prompt}]
  response = client.chat.completions.create(
      model="gpt-4o",
      messages=message,
      n=1,
      max_tokens=max_tokens,
      temperature=temperature,
      frequency_penalty=frequency_penalty
  )
  answer = response.choices[0].message.content
  print(f'오답에 따른 재 정답: {answer}')

  return answer

file_path = './2023_11_KICE.json'

with open(file_path, 'r', encoding='utf-8') as f:
  json_data = json.load(f)

total_score = 0
for data in json_data:
  prompt = f"다음 텍스트를 읽고, 질문에 대한 정답을 선택해 주세요:\n\n{data['paragraph']}\n\n"

  problems = data['problems']
  for problem in problems:
    prompt = f"질문: {problem['question']}\n"

    if 'question_plus' in problem:
      prompt += f"질문에 대한 추가적인 정보 입니다.\n{problem['question_plus']}\n\n"

    prompt += "이제 아래부터 정답을 고를 선택사항들 입니다.\n"
    for idx, choice in enumerate(problem['choices']):
      prompt += f"{idx + 1}. {choice}\n"

    prompt += "\n정답은 무엇인가요? (정답 번호를 선택해 주세요)\n\n"

    # [MYCODE] 정답을 선택할 수 있는 기회를 3번을 준다.
    for idx in range(0, 3):
      predict_answer = prediction(prompt)
      predict_answer = predict_answer[:1]
      predict_answer = int(predict_answer)
      answer = problem['answer']

      if answer == predict_answer:
        score = problem['score']
        total_score += score
        break
      else:
        predict_answer = predict_again(prompt, predict_answer)
        predict_answer = predict_answer[:1]
        predict_answer = int(predict_answer)
        if answer == predict_answer:
          score = problem['score']
          total_score += score
          break

print(f'총 점수는 {total_score} 입니다.')



gtp가 추론한 정답: 5
오답에 따른 재 정답: 5
gtp가 추론한 정답: 5
오답에 따른 재 정답: 1
gtp가 추론한 정답: 5
오답에 따른 재 정답: 1
gtp가 추론한 정답: 5
gtp가 추론한 정답: 1
gtp가 추론한 정답: 4
gtp가 추론한 정답: 1
오답에 따른 재 정답: 1
gtp가 추론한 정답: 1
오답에 따른 재 정답: 1
gtp가 추론한 정답: 1
오답에 따른 재 정답: 1
gtp가 추론한 정답: 2
오답에 따른 재 정답: 3
gtp가 추론한 정답: 2
gtp가 추론한 정답: 3
오답에 따른 재 정답: 5
gtp가 추론한 정답: 2
gtp가 추론한 정답: 4
gtp가 추론한 정답: 5
gtp가 추론한 정답: 3
오답에 따른 재 정답: 2
gtp가 추론한 정답: 5
gtp가 추론한 정답: 3
gtp가 추론한 정답: 5
오답에 따른 재 정답: 5
gtp가 추론한 정답: 5
오답에 따른 재 정답: 5
gtp가 추론한 정답: 5
오답에 따른 재 정답: 5
gtp가 추론한 정답: 3
오답에 따른 재 정답: 4
gtp가 추론한 정답: 1
gtp가 추론한 정답: 1
오답에 따른 재 정답: 4
gtp가 추론한 정답: 3
gtp가 추론한 정답: 1
오답에 따른 재 정답: 3
gtp가 추론한 정답: 4.
오답에 따른 재 정답: 5
gtp가 추론한 정답: 1
gtp가 추론한 정답: 5
gtp가 추론한 정답: 3
gtp가 추론한 정답: 4
오답에 따른 재 정답: 3
gtp가 추론한 정답: 4
gtp가 추론한 정답: 2
오답에 따른 재 정답: 1
gtp가 추론한 정답: 5
gtp가 추론한 정답: 1
gtp가 추론한 정답: 2
gtp가 추론한 정답: 3
오답에 따른 재 정답: 1
gtp가 추론한 정답: 4
gtp가 추론한 정답: 5
오답에 따른 재 정답: 2
gtp가 추론한 정답: 3
gtp가 추론한 정답: 2
오답에 따른 재 정답: 3
gtp가 추론한 정답: 2
오답에 따른 재 정답: 3
gtp가 추론한 정답: 2
오답에 따른 재 정답: 3
gtp가 추론한 